In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)
from keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D
from keras.models import Sequential

2023-06-14 20:12:10.130649: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 20:12:10.303807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 20:12:10.305136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 20:12:11.817305: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
indices_train = np.load('../data/indices_train.npy')
# indices_test = np.load('../data/indices_test.npy')

In [17]:
expanded_indices_train = np.repeat(indices_train, 25, axis=0)
print(expanded_indices_train.shape)

(874400, 4)


In [18]:
offsets = np.array([(i, j) for i in range(-2, 3) for j in range(-2, 3)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], 25) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], 25) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

In [23]:
X_full = np.stack(list(map(lambda index: np.load(f'../data/images_train/image_{index:03d}.npy'), range(40))))
y = np.stack(list(map(lambda index: np.load(f'../data/masks_train/mask_{index:03d}.npy'), range(40))))

In [51]:
X_train = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
y_train = y[indices_train[:, 0], 0, indices_train[:, 2], indices_train[:, 3]]

In [39]:
reshaped_X_train = np.reshape(X_train, ((X_train.shape[0] // 25), 5, 5, -1))

In [43]:
# Calculate the mean and standard deviation of the data
mean = np.mean(reshaped_X_train)
std = np.std(reshaped_X_train)

# Normalize the data
normalized_X_train = (reshaped_X_train - mean) / std

In [55]:
model = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])

In [56]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 3, 3, 3)           273       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 3)          0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 3)                 0         
                                                                 
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 324
Trainable params: 324
Non-trainable params: 0
________________________________________________________

In [57]:
model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])

In [59]:
history = model.fit(normalized_X_train, 
                    y_train, 
                    epochs=100, 
                    batch_size=256, 
                    validation_split=0.1)

Epoch 1/100
123/123 [==============================] - 1s 6ms/step - loss: 4.4925 - mean_absolute_error: 4.4925 - val_loss: 4.3542 - val_mean_absolute_error: 4.3542
Epoch 2/100
123/123 [==============================] - 1s 5ms/step - loss: 4.4874 - mean_absolute_error: 4.4874 - val_loss: 4.3513 - val_mean_absolute_error: 4.3513
Epoch 3/100
123/123 [==============================] - 1s 5ms/step - loss: 4.4857 - mean_absolute_error: 4.4857 - val_loss: 4.3497 - val_mean_absolute_error: 4.3497
Epoch 4/100
123/123 [==============================] - 1s 4ms/step - loss: 4.4833 - mean_absolute_error: 4.4833 - val_loss: 4.3500 - val_mean_absolute_error: 4.3500
Epoch 5/100
123/123 [==============================] - 1s 4ms/step - loss: 4.4827 - mean_absolute_error: 4.4827 - val_loss: 4.3460 - val_mean_absolute_error: 4.3460
Epoch 6/100
123/123 [==============================] - 1s 5ms/step - loss: 4.4811 - mean_absolute_error: 4.4811 - val_loss: 4.3428 - val_mean_absolute_error: 4.3428
Epoch 7/10